# Convert Tubes To Images: itk::Spatial Object bug Notebook

This notebook contains a few examples of how to call wrapped methods in itk and tubeTK ITK. The primary aim is to show a bug in TubeTK python wrapping for itk::SpatialObjects.

If ITK is not installed in your python environment, you need to define the environment variable `ITK_BUILD_DIR` that contains the path to where ITK was built.

We need to find the directory in which TubeTK was build. This is required to find the path to the testing data, and may be also required to find the TubeTK library paths if your python environment does not include it.
The environment variable `TubeTK_BUILD_DIR` needs to be defined.

In [6]:
import os
import sys
import numpy

In [7]:
# Path for TubeTK libs and bin
#Values takend from TubeTK launcher

sys.path.append("C:/src/TubeTK_Python_ITK/TubeTK-build/lib/")
sys.path.append("C:/src/TubeTK_Python_ITK/TubeTK-build/lib/Release")

In [8]:
# Setting TubeTK Build Directory

TubeTK_BUILD_DIR=None
if 'TubeTK_BUILD_DIR' in os.environ:
    TubeTK_BUILD_DIR = os.environ['TubeTK_BUILD_DIR']
else:
    print('TubeTK_BUILD_DIR not found!')
    print('  Set environment variable')
    os.environ["TubeTK_BUILD_DIR"] = "C:/src/TubeTK_Python_ITK/TubeTK-build"
    TubeTK_BUILD_DIR = os.environ["TubeTK_BUILD_DIR"]
    #sys.exit( 1 )
    
if not os.path.exists(TubeTK_BUILD_DIR):
    print('TubeTK_BUILD_DIR set by directory not found!')
    print('  TubeTK_BUILD_DIR = ' + TubeTK_BUILD_DIR )
    sys.exit(1)
    


In [9]:
try:
    import itk
except:
    ITK_BUILD_DIR = None
    if 'ITK_BUILD_DIR' in os.environ:
        ITK_BUILD_DIR = os.environ['ITK_BUILD_DIR']
    else:
        print('ITK_BUILD_DIR not found!')
        print('  Set environment variable')
        os.environ["ITK_BUILD_DIR"] = "C:/src/TubeTK_Python_R/ITK-build"
        ITK_BUILD_DIR = os.environ["ITK_BUILD_DIR"]
        #sys.exit( 1 )

    if not os.path.exists(ITK_BUILD_DIR):
        print('ITK_BUILD_DIR set by directory not found!')
        print('  ITK_BUIDL_DIR = ' + ITK_BUILD_DIR )
        sys.exit(1)
    # Append ITK libs
    sys.path.append("C:/src/TubeTK_Python_ITK/ITK-build/Wrapping/Generators/Python/Release")
    sys.path.append("C:/src/TubeTK_Python_ITK/ITK-build/lib/Release")
    sys.path.append("C:/src/TubeTK_Python_ITK/ITK-build/lib")
    
    # Append TubeTK libs
    sys.path.append("C:/src/TubeTK_Python_ITK/TubeTK-build/ITKModules/TubeTKITK-build/Wrapping/Generators/Python/Release")
    import itk

In [10]:
from itk import TubeTKITK as itktube


Initialization

In [11]:
Dimension = 3
PixelType = itk.UC

sampleTubeFileName = os.path.join(TubeTK_BUILD_DIR, 'MIDAS_Data\Branch-truth.tre')
templateImageFileName = os.path.join(TubeTK_BUILD_DIR, 'MIDAS_Data\Branch.n010.mha')
outputImageFileName = os.path.join(TubeTK_BUILD_DIR, 'Temporary\\testOutput.mha')

ImageType = itk.Image[PixelType, Dimension]
SpatialObjectType = itk.SpatialObject[Dimension]

Reading the sample tube file and creating a sample spatial object group object

In [12]:
TubeFileReaderType = itk.SpatialObjectReader[Dimension]
tubeFileReader = TubeFileReaderType.New()
tubeFileReader.SetFileName(sampleTubeFileName)
tubeFileReader.Update()

sampleSpatialObjectGroup = tubeFileReader.GetGroup()

Reading the sample image file and creating an image object

In [14]:
ImageReaderType = itk.ImageFileReader[ImageType]

imageReader = ImageReaderType.New()
imageReader.SetFileName(templateImageFileName)
imageReader.Update()

image = imageReader.GetOutput()

itkImageFileReaderIUC3: 0.000000itkImageFileReaderIUC3: 1.000000

Testing ITK filter using itk spatial object. There are two such filters 
1. itkSpatialObjectToImageFilter  (Wrapped)
2. itkSpatialObjectToPointSetFilter  (Not Wrapped)

In [15]:
# ITK Filter using ITK Spatial Object: OK

s2iType = itk.SpatialObjectToImageFilter[SpatialObjectType, ImageType]
s2i = s2iType.New()
s2i.SetInput(sampleSpatialObjectGroup)
s2i.Update()

#Save the output image
ImageWriterType = itk.ImageFileWriter[ImageType]
imageWriter = ImageWriterType.New()
imageWriter.SetInput(s2i.GetOutput())
imageWriter.SetFileName(outputImageFileName)
imageWriter.Update()


itkSpatialObjectToImageFilterSO3IUC3: 0.000000itkSpatialObjectToImageFilterSO3IUC3: 0.010000itkSpatialObjectToImageFilterSO3IUC3: 0.020000itkSpatialObjectToImageFilterSO3IUC3: 0.030000itkSpatialObjectToImageFilterSO3IUC3: 0.040000itkSpatialObjectToImageFilterSO3IUC3: 0.050000itkSpatialObjectToImageFilterSO3IUC3: 0.060000itkSpatialObjectToImageFilterSO3IUC3: 0.070000itkSpatialObjectToImageFilterSO3IUC3: 0.080000itkSpatialObjectToImageFilterSO3IUC3: 0.090000itkSpatialObjectToImageFilterSO3IUC3: 0.100000itkSpatialObjectToImageFilterSO3IUC3: 0.110000itkSpatialObjectToImageFilterSO3IUC3: 0.120000itkSpatialObjectToImageFilterSO3IUC3: 0.130000itkSpatialObjectToImageFilterSO3IUC3: 0.140000itkSpatialObjectToImageFilterSO3IUC3: 0.150000itkSpatialObjectToImageFilterSO3IUC3: 0.160000itkSpatialObjectToImageFilterSO3IUC3: 0.170000itkSpatialObjectToImageFilterSO3IUC3: 0.180000itkSpatialObjectToImageFilterSO3IUC3: 0.190000itkSpatialObjectToImageFilterSO3IUC3: 0.200000itkSpatialObjectToImageFilterSO3IU

Testing TubeTK filter using itk Spatial Object

In [20]:
# TubeTK Filter using ITK Spatial Object: NOT OK
SpatialObject = SpatialObjectType.New()

#verify sampleSpatialObjectGroup
print (isinstance(sampleSpatialObjectGroup, SpatialObjectType))

TubesToImageFilterType = itktube.ConvertTubesToImage[Dimension, PixelType]

tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetTemplateImage(image)
#tubesToImageFilter.SetInput(sampleSpatialObjectGroup) # //Expected to work, but is not working
tubesToImageFilter.SetInput(SpatialObject) # //Expected to work, but is not working
tubesToImageFilter.Update()



# Another TubeTK Filter

FilterType = itktube.ComputeTubeFlyThroughImage[PixelType, Dimension]
Filter = FilterType.New()
Filter.SetInputImage(image)
Filter.SetTubeId(0)
Filter.SetInput(sampleSpatialObjectGroup) # //Expected to work, but is not working
Filter.Update()


True


TypeError: in method 'tubeConvertTubesToImage3UC_SetInput', argument 2 of type 'itk::SpatialObject< 3 > const *'